In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
# For Data
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

#from Data.historical_data import Historical_data
import pandas as pd
from Preprocessing.preprocessing import preprocessing
from Models.LSTM.lightningLSTM import LightningLSTM
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from Preprocessing.dataclasses import StockPriceDataModule, StockDataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar, StochasticWeightAveraging
import matplotlib.pyplot as plt
import numpy as np 
import numpy as np
from tqdm import tqdm
from hyperopt import fmin, tpe, hp
import time
from pytorch_lightning.loggers import CSVLogger

In [3]:
df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')

In [4]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_checkpoint3",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

#logger = TensorBoardLogger("logs/", name="lightninglstm")

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=2)

progressbar = TQDMProgressBar(refresh_rate=100)


In [6]:
def objective(params):

    pbar.update(1)

    # Set the hyperparameters, batch size, and sequence length
    LEARNING_RATE = params['learning_rate']
    BATCH_SIZE = params['batch_size']
    SEQUENCE_LENGTH = params['sequence_length']
    HIDDEN_SIZE = params['hidden_size']
    NUM_LAYERS = params['num_layers']
    
    data_train, data_test, _ = preprocessing(df=df, lag=1, sequence_length=SEQUENCE_LENGTH, dif_all=True, train_size=0.99)

    # Create the data module with the given batch size and sequence length
    data_module = StockPriceDataModule(train_sequence=data_train, test_sequence=data_test, batch_size=BATCH_SIZE, num_workers=4)
    data_module.setup()

    # Create the Lightning module with the given hyperparameters
    model = LightningLSTM(input_size=9, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, learning_rate=LEARNING_RATE)

    # Train the model for a fixed number of epochs and measure the training time
    start_time = time.time()
    
    trainer = pl.Trainer(max_epochs=1, callbacks=[progressbar], accelerator="gpu", devices=1)
    trainer.fit(model, data_module)

    training_time = time.time() - start_time

    # Evaluate the model on the test set and return the loss
    test_loss = trainer.test(model, data_module)
    print(test_loss)
    # Define the column titles for the hyperparameter log
    log_columns = ['learning_rate', 'batch_size', 'sequence_length', 'hidden_size', 'num_layers', 'test_loss', 'training_time']

    # Log the performance for the current hyperparameter configuration
    log_dict = {
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'sequence_length': SEQUENCE_LENGTH,
        'hidden_size': HIDDEN_SIZE,
        'num_layers': NUM_LAYERS,
        'test_loss': test_loss,
        'training_time': training_time
    }

    # Save the results to a text file
    with open('results.txt', 'a') as f:
        f.write(','.join([str(log_dict[col]) for col in log_columns]) + '\n')

    # Return the test loss and set the status to 'ok'
    return {'loss': test_loss, 'status': 'ok'}

space = {
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128]),
    'sequence_length': hp.choice('sequence_length', [32, 64, 128, 256]),  
    'hidden_size': hp.choice('hidden_size', [64, 80, 96, 112, 128,256, 512]),
    'num_layers': hp.choice('num_layers', [1, 2, 3])
}

# Set the maximum number of evaluations for the hyperparameter search
max_evals = 2

# Perform the hyperparameter search using the TPE algorithm
pbar = tqdm(total=max_evals)

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, rstate=np.random.default_rng (42))

pbar.close()

best

  5%|▌         | 1/20 [03:00<57:13, 180.70s/it]

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | model     | LSTM    | 257 K 
1 | criterion | MSELoss | 0     
--------------------------------------
257 K     Trainable params
0         Non-trainable params
257 K     Total params
1.031     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.7499390840530396     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7499390840530396}]                   
  0%|          | 0/20 [01:04<?, ?trial/s, best loss=?]


                                   job exception: dictionary update sequence element #0 has length 1; 2 is required



  0%|          | 0/20 [01:04<?, ?trial/s, best loss=?]


ValueError: dictionary update sequence element #0 has length 1; 2 is required